In [1]:
"""
@author: Philip Ruthig, PhD Student at Leipzig University & MPI for Cognitive and Brain Sciences.
This script 3d tiff files as input and annotates round structures in them (e.g. cell centres, cell nuclei).
Afterward, a spatial cell center distribution is calculated and the results are saved.
"""

import numpy as np
import scipy.ndimage as ndi
import scipy.signal as sig
import math as m
from skimage.feature import peak_local_max
import skimage.morphology as morph
import tifffile as tf


c:\Users\Hoellenmaschine2\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
def gaborkernel(edge_length,sigma, freq, phase, radius, z_scale_factor):
    '''
    Returns two gabor kernels (real and imaginary) for spheroid detection. Can be convolved with an image (using scipy.signal.fftconvolve)
    
    edge_length: edge length of the kernel array
    sigma: sigma of gaussian part of the gabor kernel
    freq: frequency of the complex plane wave
    phase: phase displacement in pixels
    radius: radius of how much the gaussian wave is displaced from the origin
    z_scale_factor: factor of how much z axis is compressed. 1 for isotropic data
    '''
    if edge_length%2==0:
        edge_length = edge_length+1
    
    size_z=np.arange(0,edge_length,1)
    size_y=np.arange(0,edge_length,1)
    size_x=np.arange(0,edge_length,1)
    
    z,y,x = np.meshgrid(size_z, size_y, size_x)
    z,y,x = z-(len(size_z)//2),y-(len(size_y)//2),x-(len(size_x)//2)
    y = y*z_scale_factor
    
    A = (2*m.pi*sigma**2)
    r = np.sqrt(np.power(z,2) + np. power(y,2) + np.power(x,2))
    
    kernel_real = (1/A)*np.exp(-1*m.pi*((np.power((r-radius),2))/sigma**2)) * (np.cos((freq*(r-radius)*2*m.pi+phase)))
    kernel_imag = (1/A)*np.exp(-1*m.pi*((np.power((r-radius),2))/sigma**2)) * (np.sin((freq*(r-radius)*2*m.pi+phase)))
            
    #inverting kernels
    kernel_real = kernel_real*(-1)
    kernel_imag = kernel_imag*(-1)
        
    return kernel_real, kernel_imag

def cell_centre_distribution(bool_input,reach,sparsity_factor=1):
    '''
    Computes a mean distribution of cells in a given boolean array in a given edge length cube. Cycles through all TRUE pixels
    and checks the surroundings in a cube with the edge length "reach".

    bool_input: boolean numpy array input
    reach: edge length of the cube
    sparsity_factor: subsampling factor. 1 means every cell, 10 every 10th cell
    '''

    struct = np.zeros((3,3,3))
    struct[1,1,1] = 1
    
    centres_labeled, n_cells_labeled = ndi.label(bool_input, structure=struct)

    invalid_area_mask = np.ones_like(bool_input,dtype="bool") #define valid part of data - exclude outer rim
    invalid_area_mask[0:reach//2,:,:] = False
    invalid_area_mask[-reach//2:,:,:] = False
    invalid_area_mask[:,-reach//2:,:] = False
    invalid_area_mask[:,0:reach//2,:] = False
    invalid_area_mask[:,:,-reach//2:] = False
    invalid_area_mask[:,:,0:reach//2] = False
    
    for i in range(0,n_cells_labeled,sparsity_factor): #iterate over all cells
        if i == 0: #initialize analysis
            resultarray = np.zeros((reach,reach,reach)) #initialize results array
            n_valid_cells = 0
            n_invalid_cells = 0
            continue

        clocz,clocy,clocx = np.nonzero(centres_labeled==i) #get active cell coordinates
            
        if invalid_area_mask[clocz[0],clocy[0],clocx[0]]==True:
            n_valid_cells += 1
            tmpdata = bool_input[clocz[0]-reach//2:clocz[0]+reach//2,clocy[0]-reach//2:clocy[0]+reach//2,clocx[0]-reach//2:clocx[0]+reach//2]
        else:
            n_invalid_cells += 1
            continue
        
        resultarray = resultarray + tmpdata #add tmp data to complete results array
        
    resultarray[reach//2,reach//2,reach//2] = 0 #delete reference cell
    return resultarray

In [3]:
### tuple of image names to be processed
img_paths = (
r"data_from_bioimg_arx\PR008_L.tif",
r"data_from_bioimg_arx\PR008_L.tif",
r"data_from_bioimg_arx\PR008_R.tif",
r"data_from_bioimg_arx\PR009_L.tif",
r"data_from_bioimg_arx\PR009_R.tif",
r"data_from_bioimg_arx\PR010_L.tif",
r"data_from_bioimg_arx\PR010_R.tif",
r"data_from_bioimg_arx\PR012_L.tif",
r"data_from_bioimg_arx\PR012_R.tif",
r"data_from_bioimg_arx\PR013_L.tif",
r"data_from_bioimg_arx\PR013_R.tif",
r"data_from_bioimg_arx\PR014_L.tif",
r"data_from_bioimg_arx\PR014_R.tif",
r"data_from_bioimg_arx\PR001_L.tif",
r"data_from_bioimg_arx\PR001_R.tif",
r"data_from_bioimg_arx\PR003_L.tif",
r"data_from_bioimg_arx\PR003_R.tif",
r"data_from_bioimg_arx\PR005_L.tif",
r"data_from_bioimg_arx\PR005_R.tif",
r"data_from_bioimg_arx\PR006_L.tif",
r"data_from_bioimg_arx\PR006_R.tif",
r"data_from_bioimg_arx\PR007_L.tif",
r"data_from_bioimg_arx\PR007_R.tif",)

save_path     = r"processed\\"  #path to results folder

### variables that will need tweaking depending on the image being processed
gauss_sigma   = 0.7   #sigma for gaussian blurred image

### gabor filter variables. need to be adapted to cell size, data & quality
hucd_edge          = 40   #gabor kernel edge length (x=y=z)
hucd_gabor_freq    = 1/10 #frequency of wave component
hucd_gabor_phase   = 3.7  #wave component offset
hucd_gabor_sigma   = 8   #gaussian deviation #7
hucd_gabor_radius  = 9    #donut radius       #10  
hucd_z_factor      = 7  #factor of how much the z axis is compressed in microscopy data. 1 for isotropic data
hucd_tissue_thresh = 250  #intensity threshold to check cells are in tissue
hucd_r_maxima      = 6    #radius of footprint for detecting local maxima in the gabor filtered image #6
hucd_min_distance  = 4    #min distance between local maxima #4

### data analysis variables
reach = 100 #number of surrounding pixels taken into account for cell distribution analysis
sparsity_factor = 50 #subsampling for cell distribution analysis

### initializing variables and structuring elements
footprint_maxima = morph.ball(6)   # structuring element for detecting local maxima in gabor annulus filtered image

In [4]:
for img_path in img_paths:
    # open image
    img_file=tf.imread(img_path)
    img_shape = img_file.shape
    hucd_img = img_file[:,1,:,:]

    # apply gaussian
    hucd_img = ndi.filters.gaussian_filter(hucd_img, sigma=gauss_sigma)

    # creating filter kernel
    hucd_gabor_kernel_real, hucd_gabor_kernel_imag = gaborkernel(
                                                        edge_length=hucd_edge,
                                                        sigma=hucd_gabor_sigma,
                                                        freq=hucd_gabor_freq,
                                                        radius=hucd_gabor_radius,
                                                        z_scale_factor=hucd_z_factor,
                                                        phase=hucd_gabor_phase
                                                        )
    
    # applying gabor filter
    hucd_gaborimg_real = sig.fftconvolve(hucd_img, hucd_gabor_kernel_real, mode="same")

    print("measuring maxima")

    # measure maxima
    hucd_centers = peak_local_max(
                            image=hucd_gaborimg_real,
                            min_distance=hucd_min_distance,
                            # indices=False,
                            footprint=footprint_maxima,
                            exclude_border=0,
                            )

    print("puttig maxima into images")

    # put maxima into image
    hucd_centers_3d = np.zeros_like(hucd_gaborimg_real,dtype='bool')

    for id in hucd_centers:
        hucd_centers_3d[id[0],id[1],id[2]]=True


    # threshold centers according to tissue background intensity
    hucd_centers_3d[hucd_img<hucd_tissue_thresh]=0

    print("saving images")
    # save as bool
    tf.imwrite(save_path + img_path[21:28] + r"_hucd_centers.tif",hucd_centers_3d)

    print("do cell centre analysis")

    # do cell centre analysis
    hucd_resultarray = cell_centre_distribution(bool_input=hucd_centers_3d,reach=reach, sparsity_factor=sparsity_factor)

    print("save resultarray")

    # save resultarray as well
    tf.imwrite(save_path + img_path[21:28] + r"_hucd_center_dist.tif",hucd_resultarray)

C:\Users\Hoellenmaschine2\AppData\Local\Temp\ipykernel_12476\2382627510.py:8: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  hucd_img = ndi.filters.gaussian_filter(hucd_img, sigma=gauss_sigma)


measuring maxima
